In [1]:
!pip install epitopepredict

     |████████████████████████████████| 10.8 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 15.4 MB 45.7 MB/s eta 0:00:01
     |████████████████████████████████| 166 kB 53.4 MB/s eta 0:00:01
  Created wheel for epitopepredict: filename=epitopepredict-0.4.0-py3-none-any.whl size=5775064 sha256=0aaffd230a91e2d2def897dc2c3b872c3f7685552d3ffba0a11438c724ea42f3
  Stored in directory: /root/.cache/pip/wheels/2d/de/a0/cd72c38812400388a81a86fe308bf33b9188a705d4c12b8b23
  Created wheel for bokeh: filename=bokeh-0.12.14-py3-none-any.whl size=4878133 sha256=e200d2d784eca290a380a9ac07bc2737341f8738f5536984fd58877335592ac3
  Stored in directory: /root/.cache/pip/wheels/99/e5/46/bcf3a5973d5f12d63a4004eaaac31cead15a6b8685517a10ac
  Created wheel for wtforms-tornado: filename=wtforms_tornado-0.0.2-py3-none-any.whl size=3607 sha256=3948f8c3336a69379824474aef2a9a0d9059479432b85e9816ced85bab2a395e
  Stored in directory: /root/.cache/pip/wheels/80/91/1d/f3152a343c286e0dec85349f69fc82a7e4d

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing
from sklearn import metrics
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score, roc_auc_score
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import  math
import seaborn as sns
sns.set_context("notebook", font_scale=1.4)
import epitopepredict as ep

/kaggle/input/iqb2020/sample.csv
/kaggle/input/iqb2020/test.csv
/kaggle/input/iqb2020/train.csv
wrote config file /root/.epitopepredict/default.conf


In [3]:
from keras.layers import LSTM,Bidirectional,Embedding,BatchNormalization, GlobalMaxPooling1D, Activation, Add,Dense, Input, Dropout, Conv1D,MaxPooling1D, Flatten,AveragePooling1D, GlobalAvgPool1D
from keras.models import Model, Sequential

Using TensorFlow backend.


In [4]:
train = pd.read_csv("/kaggle/input/iqb2020/train.csv")
test = pd.read_csv("/kaggle/input/iqb2020/test.csv")
train.head()

,ID,Lable,Sequence
0,1001,-1,GGTIFDCGESCFLGTCYTKGCSCGEWKLCYGTN
1,1002,1,LEKARHGSCNYVFPA
2,1003,1,KQLKTCTSVIKLGHPCDIESCLNECFRVYNTGFATCRGDKYSQLCT...
3,1004,1,SMVAEARTCESQSHKFKGPCASDHNCASVCQTERFSGGHCRGFRRR...
4,1005,1,AKKRKKKTYTKPKKIKHKHKKVKLAVLQFYKIDGSGKVQRLRKECP...


In [5]:
train = train[train["Lable"]  != "Ôªø-1"]
train = train.astype({"Lable" : np.int8})
train.dtypes

ID           int64
Lable         int8
Sequence    object
dtype: object

In [6]:
X = np.array(train["Sequence"])
Y = np.array(train["Lable"], dtype = np.int8)
X_test  = np.array(test["Sequence"])
X.shape, Y.shape, X_test.shape

((2549,), (2549,), (368,))

In [7]:
train["Lable"].hist(xlabelsize = 10, bins = 10, grid = False)

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
def get_code_freq(df, data_name):
  
    df = df.apply(lambda x: " ".join(x))

    codes = []
    for i in df: # concatination of all codes
        codes.extend(i)

    codes_dict= Counter(codes)
    codes_dict.pop(' ') # removing white space

    print(f'Codes: {data_name}')
    print(f'Total unique codes: {len(codes_dict.keys())}')

    df = pd.DataFrame({'Code': list(codes_dict.keys()), 'Freq': list(codes_dict.values())})
    return df.sort_values('Freq', ascending=False).reset_index()[['Code', 'Freq']]
  
# train code sequence
train_code_freq = get_code_freq(train['Sequence'], 'Train')
test_code_freq  = get_code_freq(test['Sequence'], 'Test')

def plot_code_freq(df, data_name):
    plt.title(f'Code frequency: {data_name}')
    sns.barplot(x='Code', y='Freq', data=df)

Codes: Train
Total unique codes: 20
Codes: Test
Total unique codes: 20


In [9]:
plot_code_freq(train_code_freq, "Sequence")

In [10]:
plot_code_freq(test_code_freq, "Sequence")

In [11]:
for i in range(Y.shape[0]):
    Y[i] = 1 if Y[i] == 1 else 0
Y = Y.reshape(-1,1)
Y.shape

(2549, 1)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X,Y,test_size = 0.2, random_state = 1)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((2039,), (510,), (2039, 1), (510, 1))

In [13]:
codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

def show_matrix(m):
    #display a matrix
    cm = sns.light_palette("seagreen", as_cmap=True)
    display(m.style.background_gradient(cmap=cm))

def one_hot_encode(seq):
    o = list(set(codes) - set(seq))
    s = pd.DataFrame(list(seq))    
    x = pd.DataFrame(np.zeros((len(seq),len(o)),dtype=int),columns=o)    
    a = s[0].str.get_dummies(sep=',')
    a = a.join(x)
    a = a.sort_index(axis=1)
    #show_matrix(a)
    #e = a.values.flatten()
    e = a
    return e

pep='LEKARHGSCNYIVFPA'
e=one_hot_encode(pep)
print(e.shape, "One Hot Encoding")
show_matrix(e)


blosum = ep.blosum62
def blosum_encode(seq):
    #encode a peptide into blosum features
    s=list(seq)
    x = pd.DataFrame([blosum[i] for i in seq]).reset_index(drop=True)
    #show_matrix(x)
    #e = x.values.flatten()   
    e = x
    return e

def random_encode(p):
    return [np.random.randint(20) for i in pep]

e=blosum_encode(pep)
print(e.shape, "for Blosum") 
show_matrix(e)

nlf = pd.read_csv('https://raw.githubusercontent.com/dmnfarrell/epitopepredict/master/epitopepredict/mhcdata/NLF.csv',index_col=0)

def nlf_encode(seq):    
    x = pd.DataFrame([nlf[i] for i in seq]).reset_index(drop=True)  
    # show_matrix(x)
    # e = x.values.flatten()
    e = x
    return e

e=nlf_encode(pep)
print(e.shape,"NLF")
show_matrix(e)

(16, 20) One Hot Encoding


,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


(16, 24) for Blosum


,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V,B,Z,X,*
0,-1,-2,-3,-4,-1,-2,-3,-4,-3,2,4,-2,2,0,-3,-2,-1,-2,-1,1,-4,-3,-1,-4
1,-1,0,0,2,-4,2,5,-2,0,-3,-3,1,-2,-3,-1,0,-1,-3,-2,-2,1,4,-1,-4
2,-1,2,0,-1,-3,1,1,-2,-1,-3,-2,5,-1,-3,-1,0,-1,-3,-2,-2,0,1,-1,-4
3,4,-1,-2,-2,0,-1,-1,0,-2,-1,-1,-1,-1,-2,-1,1,0,-3,-2,0,-2,-1,0,-4
4,-1,5,0,-2,-3,1,0,-2,0,-3,-2,2,-1,-3,-2,-1,-1,-3,-2,-3,-1,0,-1,-4
5,-2,0,1,-1,-3,0,0,-2,8,-3,-3,-1,-2,-1,-2,-1,-2,-2,2,-3,0,0,-1,-4
6,0,-2,0,-1,-3,-2,-2,6,-2,-4,-4,-2,-3,-3,-2,0,-2,-2,-3,-3,-1,-2,-1,-4
7,1,-1,1,0,-1,0,0,0,-1,-2,-2,0,-1,-2,-1,4,1,-3,-2,-2,0,0,0,-4
8,0,-3,-3,-3,9,-3,-4,-3,-3,-1,-1,-3,-1,-2,-3,-1,-1,-2,-2,-1,-3,-3,-2,-4
9,-2,0,6,1,-3,0,0,0,1,-3,-3,0,-2,-3,-2,1,0,-4,-2,-3,3,0,-1,-4


(16, 18) NLF


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1.29,1.21,0.25,0.96,0.18,0.06,0.04,0,0.09,0.26,0.18,0.05,0,0.11,0.01,0.15,0.14,0.02
1,1.56,0.48,0.87,0.02,0.07,0.13,0.22,0.15,0.09,0.1,0.04,0.05,0.12,0.28,0.03,0.09,0.06,0.02
2,2.03,0.26,1.22,0.98,0.05,0.32,0.1,0.73,0.11,0.19,0.14,0.02,0.03,0.02,0.05,0.01,0.01,0.02
3,0.42,2.07,0.67,0.01,1.1,0.32,0.2,0.09,0.2,0.09,0.11,0.15,0.01,0.06,0.02,0.16,0.07,0.03
4,1.65,1.4,0.01,0.88,0.08,0.07,0.6,0.53,0.1,0.01,0.09,0.07,0.09,0.08,0.03,0.09,0.03,0.02
5,0.13,1.5,1.22,0.52,1.14,0.45,0.13,0.04,0.1,0.07,0.11,0.13,0.06,0.07,0.01,0.16,0.06,0.03
6,1.32,2.05,0.6,0.31,0.61,0.58,0,0.3,0.44,0.14,0.18,0.12,0.12,0.01,0.06,0,0.04,0.01
7,1.47,1.11,0.27,0.13,0.15,0.22,0.09,0.05,0.05,0.14,0.3,0.01,0.16,0.03,0.01,0.06,0.05,0.06
8,2.7,0.32,1.19,1.37,0.04,0.18,0.64,0.21,0.26,0.35,0.02,0.11,0.05,0.04,0.03,0.1,0.04,0.04
9,1.68,0.3,0.49,0.15,0.09,0.59,0.06,0.02,0.14,0,0.14,0.09,0.08,0.14,0.11,0.01,0.01,0.01


In [14]:
def encoding(x_train):
    enc_1_train = np.zeros((x_train.shape[0],100,20))
    enc_2_train = np.zeros((x_train.shape[0],100,18))
    enc_3_train = np.zeros((x_train.shape[0],100,24))*-4
    for i in tqdm(range(x_train.shape[0])):
        e3 = blosum_encode(x_train[i])
        e2 = nlf_encode(x_train[i])
        e1 = one_hot_encode(x_train[i])
        enc_1_train[i,0:e1.shape[0],:], enc_2_train[i,0:e2.shape[0],:], enc_3_train[i,0:e3.shape[0],:] = e1, e2, e3
    return enc_1_train,enc_2_train,enc_3_train

In [15]:
enc_ohe_train, enc_nlf_train, enc_blos_train = encoding(x_train)
enc_ohe_test,  enc_nlf_test,  enc_blos_test  = encoding(X_test)
enc_ohe_val,   enc_nlf_val,   enc_blos_val   = encoding(x_val)


In [16]:
enc_ohe_test.shape, enc_nlf_test.shape, enc_blos_test.shape, y_train.shape, y_val.shape

((368, 100, 20), (368, 100, 18), (368, 100, 24), (2039, 1), (510, 1))

In [17]:
def run(model,x_train,y_train,val,n = 1, epoch = 10, opti = "Adam", batch_size = 50):
    model_ = [model(x_train.shape[1:]) for i in range(n)]
    [model_[i].compile(loss='binary_crossentropy', optimizer = opti, metrics=['accuracy']) for i in range(n)]
    model_[0].summary()
    return [model_[i].fit(x_train,y_train, epochs = epoch, batch_size = batch_size,validation_data = val, verbose = 2) for i in range(n)]

In [18]:
def modelA(shape, n1 = 500, n2 = 130, f1 = 19, f2 = 13):
    model = Sequential()
    model.add(Conv1D(n1, (f1),activation = "relu", padding = "same", input_shape = shape))
    model.add(Conv1D(n2, (f2),activation = "relu", padding = "same"))
    model.add(GlobalAvgPool1D())
    model.add(Dense(1,activation = "sigmoid"))
    return model

In [19]:
model_A_OHE = run(modelA, enc_ohe_train, y_train, (enc_ohe_val, y_val)) 
model_A_NLF = run(modelA, enc_nlf_train, y_train, (enc_nlf_val, y_val))
model_A_BLO = run(modelA, enc_blos_train, y_train, (enc_blos_val, y_val))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 100, 500)          190500    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 100, 130)          845130    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 130)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 131       
Total params: 1,035,761
Trainable params: 1,035,761
Non-trainable params: 0
_________________________________________________________________
Train on 2039 samples, validate on 510 samples
Epoch 1/10
 - 4s - loss: 0.6884 - accuracy: 0.5061 - val_loss: 0.6703 - val_accuracy: 0.4843
Epoch 2/10
 - 0s - loss: 0.6472 - accuracy: 0.5777 - val_loss: 0.6236 - val_accuracy: 0.6255
Epoch 3/1

In [20]:
import matplotlib.pyplot as plt
history = model_A_OHE[0]
fig = plt.figure(figsize=(72,72))
name_title = ['Loss','Accuracy']
for i in range(0,2):
    ax=fig.add_subplot(8,8,i+1)
    plt.plot(history.history[list(history.history.keys())[i]], label = list(history.history.keys())[i] )
    plt.plot(history.history[list(history.history.keys())[i+2]],label = list(history.history.keys())[i+2] )
    plt.xlabel('Epochs', fontsize=18)
    plt.ylabel(name_title[i], fontsize=18)
    plt.legend()
    plt.show()


In [21]:
def modelB(shape):
    model = Sequential()
    model.add(Conv1D(500, (19),activation = "relu", padding = "same", input_shape = shape))
    model.add(Bidirectional(LSTM(350, return_sequences= True)))
    model.add(GlobalAvgPool1D())
    model.add(Dense(32,activation = "relu"))    
    model.add(Dense(1,activation = "sigmoid"))
    return model

In [22]:
model_B_OHE = run(modelB, enc_ohe_train, y_train, (enc_ohe_val, y_val)) 
model_B_NLF = run(modelB, enc_nlf_train, y_train, (enc_nlf_val, y_val))
model_B_BLO = run(modelB, enc_blos_train, y_train, (enc_blos_val, y_val))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 100, 500)          190500    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 700)          2382800   
_________________________________________________________________
global_average_pooling1d_4 ( (None, 700)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                22432     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 2,595,765
Trainable params: 2,595,765
Non-trainable params: 0
_________________________________________________________________
Train on 2039 samples, validate on 510 samples
Epoch 1/10
 - 9s - loss: 0.6331 - accuracy: 0.6400 - val_loss: 

In [23]:
import matplotlib.pyplot as plt
def show_result(history):
    fig = plt.figure(figsize = (72,72))
    history = history[0]
    name_title = ['Loss','Accuracy']
    for i in range(0,2):
        ax=fig.add_subplot(8,8,i+1)
        plt.plot(history.history[list(history.history.keys())[i]], label = list(history.history.keys())[i] )
        plt.plot(history.history[list(history.history.keys())[i+2]],label = list(history.history.keys())[i+2] )
        plt.xlabel('Epochs', fontsize=18)
        plt.ylabel(name_title[i], fontsize=18)
        plt.legend()
        plt.show()

In [24]:
show_result(model_B_OHE), show_result(model_B_NLF), show_result(model_B_BLO)

(None, None, None)

In [25]:
def modelC(shape):
    
    model = Sequential()

    model.add(Conv1D(32, kernel_size = 3, activation='relu', input_shape = shape))
    model.add(BatchNormalization())
    model.add(Conv1D(32, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv1D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.44))

    model.add(Conv1D(256, kernel_size = 4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    # model.add(Dropout(0.65))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [26]:
model_C_OHE = run(modelC, enc_ohe_train, y_train, (enc_ohe_val, y_val)) 
show_result(model_C_OHE)
model_C_NLF = run(modelC, enc_nlf_train, y_train, (enc_nlf_val, y_val))
show_result(model_C_NLF)
model_C_BLO = run(modelC, enc_blos_train, y_train, (enc_blos_val, y_val))
show_result(model_C_BLO)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_10 (Conv1D)           (None, 98, 32)            1952      
_________________________________________________________________
batch_normalization_1 (Batch (None, 98, 32)            128       
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 96, 32)            3104      
_________________________________________________________________
batch_normalization_2 (Batch (None, 96, 32)            128       
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 48, 32)            5152      
_________________________________________________________________
batch_normalization_3 (Batch (None, 48, 32)            128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 32)           

Train on 2039 samples, validate on 510 samples
Epoch 1/10
 - 2s - loss: 0.8973 - accuracy: 0.5277 - val_loss: 0.6682 - val_accuracy: 0.6294
Epoch 2/10
 - 1s - loss: 0.7032 - accuracy: 0.6498 - val_loss: 0.6580 - val_accuracy: 0.5725
Epoch 3/10
 - 1s - loss: 0.6196 - accuracy: 0.6974 - val_loss: 0.5805 - val_accuracy: 0.7000
Epoch 4/10
 - 1s - loss: 0.5256 - accuracy: 0.7518 - val_loss: 0.5311 - val_accuracy: 0.7235
Epoch 5/10
 - 1s - loss: 0.4466 - accuracy: 0.7847 - val_loss: 0.4564 - val_accuracy: 0.7804
Epoch 6/10
 - 1s - loss: 0.4185 - accuracy: 0.8185 - val_loss: 0.4857 - val_accuracy: 0.7431
Epoch 7/10
 - 1s - loss: 0.4314 - accuracy: 0.8122 - val_loss: 0.4919 - val_accuracy: 0.7725
Epoch 8/10
 - 1s - loss: 0.3479 - accuracy: 0.8480 - val_loss: 0.4234 - val_accuracy: 0.8118
Epoch 9/10
 - 1s - loss: 0.3273 - accuracy: 0.8475 - val_loss: 0.4524 - val_accuracy: 0.8059
Epoch 10/10
 - 1s - loss: 0.3071 - accuracy: 0.8632 - val_loss: 0.4750 - val_accuracy: 0.7843


In [27]:
from keras.layers import *
def modelD(shape):
    emb = Input(shape=shape)
    bi_rnn = Bidirectional(LSTM(264))(emb)
    x = Dropout(0.3)(bi_rnn)

    # softmax classifier
    x_output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=emb, outputs=x_output)

    return model

In [28]:
model_D_OHE = run(modelD, enc_ohe_train, y_train, (enc_ohe_val, y_val), epoch = 16) 
show_result(model_D_OHE)
model_D_NLF = run(modelD, enc_nlf_train, y_train, (enc_nlf_val, y_val), epoch = 16)
show_result(model_D_NLF)
model_D_BLO = run(modelD, enc_blos_train, y_train, (enc_blos_val, y_val), epoch = 16)
show_result(model_D_BLO)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 20)           0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 528)               601920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 528)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 529       
Total params: 602,449
Trainable params: 602,449
Non-trainable params: 0
_________________________________________________________________
Train on 2039 samples, validate on 510 samples
Epoch 1/16
 - 8s - loss: 0.6652 - accuracy: 0.6018 - val_loss: 0.6405 - val_accuracy: 0.6569
Epoch 2/16
 - 7s - loss: 0.6295 - accuracy: 0.6626 - val_loss: 0.5681 - val_accuracy: 0.7000
Epoch 3/16
 - 7s -

In [29]:
from keras.regularizers import l2
def residual_block(data, filters, d_rate):

    shortcut = data

    bn1 = BatchNormalization()(data)
    act1 = Activation('relu')(bn1)
    conv1 = Conv1D(filters, 1, dilation_rate=d_rate, padding='same')(act1)

    #bottleneck convolution
    bn2 = BatchNormalization()(conv1)
    act2 = Activation('relu')(bn2)
    conv2 = Conv1D(filters, 3, padding='same')(act2)

    #skip connection
    x = Add()([conv2, shortcut])
    return x

In [30]:
def modelE(shape):
    x_input = Input(shape=shape)

    #initial conv
    conv = Conv1D(90, 1, padding='same')(x_input) 

    # per-residue representation
    res1 = residual_block(conv, 90, 2)
    res2 = residual_block(res1, 90, 3)

    x = MaxPooling1D(3)(res2)
    x = Dropout(0.5)(x)

    # softmax classifier
    x = Flatten()(x)
    x_output = Dense(1, activation='sigmoid')(x)

    model2 = Model(inputs=x_input, outputs=x_output)
    return model2

In [31]:
model_E_OHE = run(modelE, enc_ohe_train, y_train, (enc_ohe_val, y_val), epoch = 16) 
show_result(model_E_OHE)
model_E_NLF = run(modelE, enc_nlf_train, y_train, (enc_nlf_val, y_val), epoch = 16)
show_result(model_E_NLF)
model_E_BLO = run(modelE, enc_blos_train, y_train, (enc_blos_val, y_val), epoch = 16)
show_result(model_E_BLO)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 100, 20)      0                                            
__________________________________________________________________________________________________
conv1d_31 (Conv1D)              (None, 100, 90)      1890        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 100, 90)      360         conv1d_31[0][0]                  
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 100, 90)      0           batch_normalization_22[0][0]     
____________________________________________________________________________________________

Train on 2039 samples, validate on 510 samples
Epoch 1/16
 - 2s - loss: 0.8542 - accuracy: 0.5787 - val_loss: 0.6832 - val_accuracy: 0.5608
Epoch 2/16
 - 0s - loss: 0.5206 - accuracy: 0.7283 - val_loss: 0.7091 - val_accuracy: 0.5000
Epoch 3/16
 - 0s - loss: 0.4044 - accuracy: 0.8151 - val_loss: 0.7870 - val_accuracy: 0.4843
Epoch 4/16
 - 0s - loss: 0.3417 - accuracy: 0.8416 - val_loss: 0.8386 - val_accuracy: 0.4863
Epoch 5/16
 - 0s - loss: 0.2880 - accuracy: 0.8813 - val_loss: 0.7999 - val_accuracy: 0.5020
Epoch 6/16
 - 0s - loss: 0.2533 - accuracy: 0.8950 - val_loss: 0.8163 - val_accuracy: 0.5118
Epoch 7/16
 - 0s - loss: 0.2276 - accuracy: 0.9083 - val_loss: 0.8534 - val_accuracy: 0.5294
Epoch 8/16
 - 0s - loss: 0.2143 - accuracy: 0.9196 - val_loss: 0.4520 - val_accuracy: 0.7765
Epoch 9/16
 - 0s - loss: 0.1625 - accuracy: 0.9416 - val_loss: 0.3969 - val_accuracy: 0.8255
Epoch 10/16
 - 0s - loss: 0.1611 - accuracy: 0.9377 - val_loss: 0.3855 - val_accuracy: 0.8275
Epoch 11/16
 - 0s - lo

In [32]:
from sklearn.linear_model import LogisticRegression
X, y = np.sum(enc_ohe_train,axis =  1) , y_train
clf = LogisticRegression(C = 10,max_iter = 1000,solver =  "newton-cg" ,warm_start = True,random_state=0).fit(X, y)
print(clf.score(np.sum(enc_ohe_val,axis =  1) , y_val))
clf.get_params(), clf.coef_

0.707843137254902


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


({'C': 10,
  'class_weight': None,
  'dual': False,
  'fit_intercept': True,
  'intercept_scaling': 1,
  'l1_ratio': None,
  'max_iter': 1000,
  'multi_class': 'auto',
  'n_jobs': None,
  'penalty': 'l2',
  'random_state': 0,
  'solver': 'newton-cg',
  'tol': 0.0001,
  'verbose': 0,
  'warm_start': True},
 array([[-0.10385761, -0.01320465, -0.12088109, -0.03387799, -0.12028869,
          0.06971054,  0.26619075, -0.11169293,  0.14322326, -0.20669673,
          0.02025966,  0.01553892, -0.06107983,  0.01693183,  0.19128099,
          0.09124813,  0.1036921 , -0.05270129, -0.61283254, -0.00605382]]))

In [33]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import numpy as np

clf = QuadraticDiscriminantAnalysis()
clf.fit(X, y)
QuadraticDiscriminantAnalysis()
print(clf.score(np.sum(enc_ohe_val,axis =  1) , y_val))

0.7705882352941177


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [34]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=  7, p = 2,weights = "distance", algorithm= "brute")
neigh.fit(X, y)
neigh.score(np.sum(enc_ohe_val,axis =  1) , y_val)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  after removing the cwd from sys.path.


0.8549019607843137

In [ ]:
header = ["ID", "LABEL"]
tmp = np.arange(3551, 3551+368).reshape(-1,1)
pd.DataFrame(np.concatenate((tmp,result), axis = 1).astype(np.int)).to_csv("result.csv",header= header, index = None)                        

In [ ]:
def calc_result(model, test, n):
        result = np.zeros((test.shape[0],1))
    for i in range(n):
        result += model[i].predict_proba(test)
    for i in range(result.shape[0]):
        result[i] = 1 if result[i] >= n/2 else -1
    return result

In [ ]:
def result_sub(result, sub = False):
    
    pd.DataFrame(np.concatenate((np.arange(3551, 3551+result.shape[0]).reshape(-1,1),result), axis = 1).astype(np.int)).to_csv("submission.csv",header= ["ID", "LABEL"], index = None)
    if sub:
        !kaggle competitions submit -c iqb2020 -f /kaggle/working/result.csv -m "Message"


In [ ]:
result_sub(calc_result(modelA, enc_ohe_test, 20 ))